In [1]:
##https://colab.research.google.com/github/trekhleb/machine-learning-experiments/blob/master/experiments/image_recognition_cnn/image_recognition_cnn.ipynb#scrollTo=ZTO-30-IMfK3
##how to feed grayscale images to pretrained models which requires 3 input channels:
##https://stackoverflow.com/questions/51995977/how-can-i-use-a-pre-trained-neural-network-with-grayscale-images
##attention ref: https://github.com/dt024/RIVF2021_fakenews

import os
import time
import datetime
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from official import nlp
import official.nlp.optimization

In [2]:
# !pip install tensorflow==2.7.3
# !pip install -q tf-models-official==2.7.0
# !pip install tensorflow_hub
# !pip install tensorflow_text
# # !pip install keras-lr-multiplier

In [3]:
# pip list -v 

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0:2], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2 Physical GPUs, 2 Logical GPU


In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
# from google.colab import files
# uploaded = files.upload()

In [8]:
from sklearn.model_selection import train_test_split

sketch = pd.read_csv('data/sketch_drawing.csv')

# var = 'Unique_expert'
var = 'Drawing_expert'
# var = 'Useful_expert'
# var = 'Elegence_expert'
# var = 'Creativity_expert'

mapping1 = {'Creativity_expert': 'Creativity',
                       'Useful_expert': 'Useful',
                       'Unique_expert': 'Unique',
                       'Drawing_expert': 'Drawing',
                       'Elegence_expert': 'Elegence',}
mapping = {'Creativity_expert': 'Creativity_expert_round',
                       'Useful_expert': 'Useful_expert_round',
                       'Unique_expert': 'Unique_expert_round',
                       'Drawing_expert': 'Drawing_expert_round',
                       'Elegence_expert': 'Elegence_expert_round',}


In [9]:
# sketch1 = pd.read_csv('data/sketch_drawing.csv')
sketch1 = pd.read_csv('data/sketch_drawing.csv')
sketch_emb = np.loadtxt('sketch emb-'+var) 
# sketchs = [sketch[i] for i in sketch.keys()]
train_df = sketch1[sketch1['set2_'+var] == 'train'] 
val_df = sketch1[sketch1['set2_'+var] == 'val'] 
test_df = sketch1[sketch1['set2_'+var] == 'test'] 

train_emb = np.array([sketch_emb[i] for i in train_df.index])
val_emb = np.array([sketch_emb[i] for i in val_df.index])
test_emb = np.array([sketch_emb[i] for i in test_df.index])

##to add
sketch1 = sketch1.astype({"text": str})
text = list(sketch1['text'])
train_text = [text[i] for i in train_df.index]
val_text = [text[i] for i in val_df.index]
test_text = [text[i] for i in test_df.index]

In [10]:
train_cost = train_df[var]
val_cost = val_df[var]
test_cost = test_df[var]

In [11]:
import os
import shutil
import logging
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal.windows import triang

def get_bin_idx(x):
    return min(int(x * np.float32(10)), 99)

def get_lds_kernel_window(kernel, ks, sigma):
    assert kernel in ['gaussian', 'triang', 'laplace']
    half_ks = (ks - 1) // 2
    if kernel == 'gaussian':
        base_kernel = [0.] * half_ks + [1.] + [0.] * half_ks
        kernel_window = gaussian_filter1d(base_kernel, sigma=sigma) / max(gaussian_filter1d(base_kernel, sigma=sigma))
    elif kernel == 'triang':
        kernel_window = triang(ks)
    else:
        laplace = lambda x: np.exp(-abs(x) / sigma) / (2. * sigma)
        kernel_window = list(map(laplace, np.arange(-half_ks, half_ks + 1))) / max(map(laplace, np.arange(-half_ks, half_ks + 1)))

    return kernel_window

In [12]:
# https://github.com/YyzHarry/imbalanced-regression''

from collections import Counter
from scipy.ndimage import convolve1d

for i in [0]:
    # preds, labels: [Ns,], "Ns" is the number of total samples
#     preds, labels = train_df, train_cost1[var]
    labels = train_cost
    # assign each label to its corresponding bin (start from 0)
    # with your defined get_bin_idx(), return bin_index_per_label: [Ns,] 
    bin_index_per_label = [get_bin_idx(label) for label in labels]

    # calculate empirical (original) label distribution: [Nb,]
    # "Nb" is the number of bins
    Nb = max(bin_index_per_label) + 1
    num_samples_of_bins = dict(Counter(bin_index_per_label))
    emp_label_dist = [num_samples_of_bins.get(i, 0) for i in range(Nb)]

    # lds_kernel_window: [ks,], here for example, we use gaussian, ks=5, sigma=2
    lds_kernel_window = get_lds_kernel_window(kernel='gaussian', ks=5, sigma=2)
    # calculate effective label distribution: [Nb,]
    eff_label_dist = convolve1d(np.array(emp_label_dist), weights=lds_kernel_window, mode='constant')

    # Use re-weighting based on effective label distribution, sample-wise weights: [Ns,]
    eff_num_per_label = [eff_label_dist[bin_idx] for bin_idx in bin_index_per_label]
    weights = [np.float32(1 / x) for x in eff_num_per_label]
    # # calculate loss
    # loss = weighted_mse_loss(preds, labels, weights=weights)                  

In [13]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
# bert_model_name = 'small_bert/bert_en_uncased_L-4_H-768_A-12'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [23]:
import tensorflow_hub as hub
import tensorflow_text as text
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [24]:
# text

In [25]:
text = list(sketch1['text'])
text_preprocessed = bert_preprocess_model(text[0:1])
bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_results = bert_model(text_preprocessed)
bert_results.keys()

dict_keys(['default', 'sequence_output', 'pooled_output', 'encoder_outputs'])

In [29]:
# bert_results

In [26]:
bert_results['encoder_outputs'][0].shape

TensorShape([1, 128, 512])

In [17]:
# text_preprocessed_train = bert_preprocess_model(train_text)
# text_preprocessed_val = bert_preprocess_model(val_text)
# text_preprocessed_test = bert_preprocess_model(test_text)

# text_test = ['this is such an amazing movie!']
# text_preprocessed = bert_preprocess_model(text_test)

# print(f'Keys       : {list(text_preprocessed.keys())}')
# print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
# print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
# print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
# print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

In [22]:
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model, Input, layers, regularizers
from keras.models import load_model
from tensorflow.keras import activations
from official.nlp import optimization 

epochs = 1000
steps_per_epoch = 56
num_train_steps = steps_per_epoch * epochs
##unique: 0.005, drawing: 0.005, elegance: 0.05, creativity: 0.05, useful: 0.1
a = 0.01
if var == 'Unique_expert' or var == 'Drawing_expert':
    a = 0.005
# elif var == 'Creativity_expert':
#     a = 0.005
# elif var == 'Useful_expert':
#     a = 0.01
num_warmup_steps = int(a*num_train_steps)
init_lr = 2e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

merge = []
text_input = Input(shape=(), dtype=tf.string, name='text')
input_tensor = Input(shape=1048, name='input')

# preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
# encoder_inputs = preprocessing_layer(text_input)
# encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
# outputs = encoder(encoder_inputs)

# sequence_output_1 = outputs['encoder_outputs'][0][:,0,:]
# sequence_output_1 = layers.BatchNormalization()(sequence_output_1)
# sequence_output_1 = layers.Activation('tanh')(sequence_output_1)
# sequence_output_2 = outputs['encoder_outputs'][1][:,0,:]
# sequence_output_2 = layers.BatchNormalization()(sequence_output_2)
# sequence_output_2 = layers.Activation('tanh')(sequence_output_2)
# sequence_output_3 = outputs['encoder_outputs'][2][:,0,:]
# sequence_output_3 = layers.BatchNormalization()(sequence_output_3)
# sequence_output_3 = layers.Activation('tanh')(sequence_output_3)
# sequence_output_4 = outputs['encoder_outputs'][3][:,0,:]
# sequence_output_4 = layers.BatchNormalization()(sequence_output_4)
# sequence_output_4 = layers.Activation('tanh')(sequence_output_4)
# sequence_output_1 = layers.Reshape((-1,sequence_output_4.shape[-1]))(sequence_output_1)
# sequence_output_2 = layers.Reshape((-1,sequence_output_4.shape[-1]))(sequence_output_2)
# sequence_output_3 = layers.Reshape((-1,sequence_output_4.shape[-1]))(sequence_output_3)
# sequence_output_4 = layers.Reshape((-1,sequence_output_4.shape[-1]))(sequence_output_4)
# # print (sequence_output_4.shape)
# sequence_output = layers.Concatenate(axis=1)([sequence_output_1, sequence_output_2, sequence_output_3, sequence_output_4])
# # print (sequence_output.shape)

model_name = 'model weight/'+var+'/new text-'+var+'-best.h5'
base_model = load_model(model_name, custom_objects={'KerasLayer':hub.KerasLayer,'AdamWeightDecay': optimizer})
for layer in base_model.layers:
    layer._name = layer._name + "_a"

sequence_output_1 = base_model.layers[2].output['encoder_outputs'][0][:,0,:]
sequence_output_1 = layers.BatchNormalization()(sequence_output_1)
sequence_output_1 = layers.Activation('tanh')(sequence_output_1)
sequence_output_2 = base_model.layers[2].output['encoder_outputs'][1][:,0,:]
sequence_output_2 = layers.BatchNormalization()(sequence_output_2)
sequence_output_2 = layers.Activation('tanh')(sequence_output_2)
sequence_output_3 = base_model.layers[2].output['encoder_outputs'][2][:,0,:]
sequence_output_3 = layers.BatchNormalization()(sequence_output_3)
sequence_output_3 = layers.Activation('tanh')(sequence_output_3)
sequence_output_4 = base_model.layers[2].output['encoder_outputs'][3][:,0,:]
sequence_output_4 = layers.BatchNormalization()(sequence_output_4)
sequence_output_4 = layers.Activation('tanh')(sequence_output_4)
sequence_output_1 = layers.Reshape((-1,sequence_output_4.shape[-1]))(sequence_output_1)
sequence_output_2 = layers.Reshape((-1,sequence_output_4.shape[-1]))(sequence_output_2)
sequence_output_3 = layers.Reshape((-1,sequence_output_4.shape[-1]))(sequence_output_3)
sequence_output_4 = layers.Reshape((-1,sequence_output_4.shape[-1]))(sequence_output_4)
# print (sequence_output_4.shape)
sequence_output = layers.Concatenate(axis=1)([sequence_output_1, sequence_output_2, sequence_output_3, sequence_output_4])
# print (sequence_output.shape)

drop_rate = 0.1
drop_rate1 = 0.3
final_hid = 64
head = 1
att_layers = 1
att_hid = 64

sketch_append = input_tensor
sketch_append = layers.Activation('tanh')(sketch_append)
sketch_append = layers.Dropout(drop_rate1)(sketch_append)

text_append =  base_model.layers[2].output['pooled_output']
text_append1 =  base_model.layers[-3].output

##With attention - sketch on text
inpAttText_key = sequence_output
inpAttText_query = sketch_append
for layer in range(att_layers):
    att_text = []
    concat_key = []
    for _ in range(head):
        img_query = layers.Dense(att_hid/head, use_bias=False)(inpAttText_query)
        img_query1 = layers.BatchNormalization()(img_query)
        img_query1 = layers.Activation('relu')(img_query1)
        # img_query1 = layers.Dropout(drop_rate1)(img_query1)
        # img_query = layers.BatchNormalization()(img_query)
        # img_query = layers.Activation('tanh')(img_query)
        img_query = layers.Dropout(drop_rate)(img_query)
        
        text_key = layers.Dense(att_hid/head, use_bias=False)(inpAttText_key)
        text_key1 = layers.BatchNormalization()(text_key)
        text_key1 = layers.Activation('relu')(text_key1)
        # text_key1 = layers.Dropout(drop_rate1)(text_key1)
        # text_key = layers.BatchNormalization()(text_key)
        # text_key = layers.Activation('tanh')(text_key)
        text_key = layers.Dropout(drop_rate)(text_key)
        
        text_value = layers.Dense(att_hid/head, use_bias=False)(inpAttText_key)
        text_value = layers.Dropout(drop_rate)(text_value)
        
        # print (img_query.shape)
        # print (text_key.shape)
        attention = layers.Dot(axes=(1,2))([img_query, text_key])
        attention = layers.Lambda(lambda x: x[0]/x[1])([attention,np.sqrt(att_hid/head)])
        attention = layers.Activation("softmax")(attention)
        # print (attention.shape)
        # print (text_value.shape)
        head_att_text = layers.Dot(axes=(1,1))([attention, text_value])
        att_text = head_att_text
        concat_key = text_key
    # att_text = Concatenate(axis=1)(att_text)
    # att_text = Dense(512, use_bias=False)(att_text)
    # att_text = Dropout(0.3)(att_text)
    # att_text = Add()([att_text, inpAttText_query])
    # att_text = LayerNormalization()(att_text)
    # att_text2 = Dense(1024,activation='relu')(att_text)
    # att_text2 = Dense(512)(att_text)
    # att_text2 = Dropout(0.1)(att_text2)
    # att_text = Add()([att_text, att_text2])
    # att_text = LayerNormalization()(att_text)

    # concat_key = Concatenate(axis=1)(concat_key)
    # concat_key = Dense(512)(concat_key)
    # concat_key = Dropout(0.3)(concat_key)
    # concat_key = Add()([concat_key, inpAttText_key])
    # concat_key = LayerNormalization()(concat_key)
    # concat_key2 = Dense(1024,activation='relu')(concat_key)
    # concat_key2 = Dense(512)(concat_key)
    # concat_key2 = Dropout(0.1)(concat_key2)
    # concat_key = Add()([concat_key, concat_key2])
    # concat_key = LayerNormalization()(concat_key)
    
    inpAttText_query = att_text
    inpAttText_key = concat_key
    
merge.append(att_text)
merge.append(img_query1)
# print (img_query1.shape)
# text_key1 = layers.Reshape((-1,256))(text_key1)
# merge.append(text_key1)
# print (text_key1.shape)
merge.append(text_append1)

l_merge = layers.Concatenate(axis=1)(merge)
l_merge = layers.Dropout(drop_rate1)(l_merge)

out4 = layers.Dense(1, activation='relu', name='out4')(l_merge)
model = Model([base_model.input,input_tensor], out4)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.MeanSquaredError(),
              metrics= tf.keras.metrics.RootMeanSquaredError())

In [23]:
##definition of callbacks
epochs = 1000
steps_per_epoch = 37
batch_size = 24

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=10,
#     patience=5,
    monitor="val_loss",
    restore_best_weights=True,
    verbose=1)

# # lr = 0.0000003
# lr = 0.00001
# def decay(epoch, steps=100):
# #     initial_lrate = 0.0001
#     initial_lrate = lr
# #     initial_lrate = 0.0000003
# #     drop = 0.9
#     drop = 1
#     epochs_drop = 4
#     lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
#     return lrate
# lr_sc = LearningRateScheduler(decay, verbose=1)

In [24]:
# num_warmup_steps = 0
# init_lr = 1.5e-5
# optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                           num_train_steps=num_train_steps,
#                                           num_warmup_steps=num_warmup_steps,
#                                           optimizer_type='adamw')
# model.compile(optimizer=optimizer,
#               loss=tf.keras.losses.MeanSquaredError(),
#               metrics= tf.keras.metrics.RootMeanSquaredError())

# early_stopping_callback = tf.keras.callbacks.EarlyStopping(
#     patience=5,
# #     patience=5,
#     monitor="val_loss",
#     restore_best_weights=True,
#     verbose=1)

In [27]:
training_history = model.fit(
    x = [tf.constant(train_text),train_emb],
#     x = x_train,
    # x = tf.constant(train_text),
    y = {'out4':np.array(train_cost)},
    sample_weight={'out4': np.array(weights)},
    epochs=epochs,
    batch_size = batch_size,
    steps_per_epoch=steps_per_epoch,
    validation_data=(
        [tf.constant(val_text),val_emb],
#                      x_val,
        # tf.constant(val_text),
        {'out4':np.array(val_cost)}),
    callbacks=[early_stopping_callback]
)

Epoch 1/1000
37/37 [==============================] - 10s 258ms/step - loss: 0.0047 - root_mean_squared_error: 0.6151 - val_loss: 0.8722 - val_root_mean_squared_error: 0.9339
Epoch 2/1000
37/37 [==============================] - 9s 255ms/step - loss: 0.0055 - root_mean_squared_error: 0.6547 - val_loss: 0.8856 - val_root_mean_squared_error: 0.9411
Epoch 3/1000
37/37 [==============================] - 10s 263ms/step - loss: 0.0057 - root_mean_squared_error: 0.6434 - val_loss: 1.0388 - val_root_mean_squared_error: 1.0192
Epoch 4/1000
37/37 [==============================] - 10s 260ms/step - loss: 0.0056 - root_mean_squared_error: 0.6513 - val_loss: 0.8817 - val_root_mean_squared_error: 0.9390
Epoch 5/1000
37/37 [==============================] - 10s 263ms/step - loss: 0.0058 - root_mean_squared_error: 0.6296 - val_loss: 0.9560 - val_root_mean_squared_error: 0.9778
Epoch 6/1000
37/37 [==============================] - 10s 265ms/step - loss: 0.0052 - root_mean_squared_error: 0.6081 - val_lo

KeyboardInterrupt: 

In [33]:
idx = 0
n = 2
# a = 0.5
output = pd.DataFrame()
# test_results0 = base_model1.predict(x_test)
# test_results0 = base_model2.predict(test_embedding)

# test_results0 = model.predict([x_test,test_embedding])
# test_results0 = model.predict(x_test)
test_results0 = model.predict([tf.constant(test_text),test_emb])
test_result= test_results0.reshape(len(test_text)).tolist()
test_cost = test_df[var]

difference_array = np.subtract(test_result, test_cost)
squared_array = np.square(difference_array)
mse = squared_array.mean()

correlation_matrix = np.corrcoef(test_cost, test_result)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

output.loc[idx,'warm up'] = a
output.loc[idx,'var'] = var
output.loc[idx,'lr'] = init_lr
output.loc[idx,'mse'] = mse
output.loc[idx,'r^2'] = r_squared

In [34]:
output

,warm up,var,lr,mse,r^2
0,0.005,Drawing_expert,0.000015,0.739586,0.359981


In [ ]:
# model.summary()

In [32]:
##save and load models
##https://stackoverflow.com/questions/46610732/how-to-freeze-some-layers-when-fine-tune-resnet50
# a = 1

# var = 'Unique_expert'
# var = 'Drawing_expert'
# var = 'Useful_expert'
# var = 'Elegence_expert'
# var = 'Creativity_expert'

model_name = 'model weight/'+var+'/attend-text1_'+var+'-best.h5'
# model_name = 'model weight/both_ratio-'+str(1)+'-'+var+'-best.h5'
# model_name = 'model weight/'+var+'/simple_joint_ratio-'+str(a)+'-'+var+'-best.h5'
# model_name = 'model weight/'+var+'/joint_lr-'+str(initial_lrate)+'-'+var+'-best.h5'
# model.save(model_name, save_format='h5')
# model.save_weights(model_name)

# from keras.models import load_model
# model = load_model(model_name)
##for text model
# model = load_model(model_name, custom_objects={'KerasLayer':hub.KerasLayer,'AdamWeightDecay': optimizer})
##for joint model
model.load_weights(model_name)

In [35]:
maxv = 0.36
idx = 0
output = pd.DataFrame()
for i in range(50):
    model_name = 'model weight/'+var+'/attend-text1_'+var+'-best.h5'
    model.load_weights(model_name)
    num_warmup_steps = 0
    init_lr = 1e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                              num_train_steps=num_train_steps,
                                              num_warmup_steps=num_warmup_steps,
                                              optimizer_type='adamw')
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics= tf.keras.metrics.RootMeanSquaredError())

    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        patience=5,
    #     patience=5,
        monitor="val_loss",
        restore_best_weights=True,
        verbose=1)
    training_history = model.fit(
        x = [tf.constant(train_text),train_emb],
    #     x = x_train,
        # x = tf.constant(train_text),
        y = {'out4':np.array(train_cost)},
        sample_weight={'out4': np.array(weights)},
        epochs=epochs,
        batch_size = batch_size,
        steps_per_epoch=steps_per_epoch,
        validation_data=(
            [tf.constant(val_text),val_emb],
    #                      x_val,
            # tf.constant(val_text),
            {'out4':np.array(val_cost)}),
        callbacks=[early_stopping_callback]
    )
    test_results0 = model.predict([tf.constant(test_text),test_emb])
    test_result= test_results0.reshape(len(test_text)).tolist()
    test_cost = test_df[var]

    difference_array = np.subtract(test_result, test_cost)
    squared_array = np.square(difference_array)
    mse = squared_array.mean()

    correlation_matrix = np.corrcoef(test_cost, test_result)
    correlation_xy = correlation_matrix[0,1]
    r_squared = correlation_xy**2

    output.loc[idx,'warm up'] = a
    output.loc[idx,'var'] = var
    output.loc[idx,'lr'] = init_lr
    output.loc[idx,'mse'] = mse
    output.loc[idx,'r^2'] = r_squared
    idx += 1
    print (idx)
    if r_squared > maxv:
        model_name1 = 'model weight/'+var+'/attend-text1_'+var+'-best.h5'
        model.save_weights(model_name1)
        maxv = r_squared
        print (r_squared)
#         if r_squared > 0.222:
#             break

Epoch 1/1000
37/37 [==============================] - 14s 275ms/step - loss: 0.0043 - root_mean_squared_error: 0.5971 - val_loss: 0.8865 - val_root_mean_squared_error: 0.9416
Epoch 2/1000
37/37 [==============================] - 9s 255ms/step - loss: 0.0050 - root_mean_squared_error: 0.6311 - val_loss: 0.9322 - val_root_mean_squared_error: 0.9655
Epoch 3/1000
37/37 [==============================] - 9s 252ms/step - loss: 0.0059 - root_mean_squared_error: 0.6595 - val_loss: 0.9753 - val_root_mean_squared_error: 0.9876
Epoch 4/1000
37/37 [==============================] - 9s 249ms/step - loss: 0.0052 - root_mean_squared_error: 0.6462 - val_loss: 0.9226 - val_root_mean_squared_error: 0.9605
Epoch 5/1000
37/37 [==============================] - 9s 251ms/step - loss: 0.0044 - root_mean_squared_error: 0.5984 - val_loss: 0.9053 - val_root_mean_squared_error: 0.9515
Epoch 6/1000
37/37 [==============================] - 10s 260ms/step - loss: 0.0047 - root_mean_squared_error: 0.6297 - val_loss:

37/37 [==============================] - 9s 255ms/step - loss: 0.0058 - root_mean_squared_error: 0.6244 - val_loss: 0.9312 - val_root_mean_squared_error: 0.9650
Epoch 3/1000
37/37 [==============================] - 10s 259ms/step - loss: 0.0047 - root_mean_squared_error: 0.6020 - val_loss: 0.9231 - val_root_mean_squared_error: 0.9608
Epoch 4/1000
37/37 [==============================] - 9s 253ms/step - loss: 0.0055 - root_mean_squared_error: 0.6102 - val_loss: 0.9229 - val_root_mean_squared_error: 0.9607
Epoch 5/1000
37/37 [==============================] - 9s 250ms/step - loss: 0.0056 - root_mean_squared_error: 0.6260 - val_loss: 0.9976 - val_root_mean_squared_error: 0.9988
Epoch 6/1000
37/37 [==============================] - 9s 255ms/step - loss: 0.0053 - root_mean_squared_error: 0.6274 - val_loss: 0.9056 - val_root_mean_squared_error: 0.9516
Epoch 00006: early stopping
11
Epoch 1/1000
37/37 [==============================] - 14s 268ms/step - loss: 0.0054 - root_mean_squared_error: 

Epoch 16/1000
37/37 [==============================] - 9s 237ms/step - loss: 0.0047 - root_mean_squared_error: 0.6128 - val_loss: 0.8956 - val_root_mean_squared_error: 0.9464
Epoch 17/1000
37/37 [==============================] - 10s 260ms/step - loss: 0.0045 - root_mean_squared_error: 0.5739 - val_loss: 0.8980 - val_root_mean_squared_error: 0.9476
Epoch 18/1000
37/37 [==============================] - 10s 258ms/step - loss: 0.0059 - root_mean_squared_error: 0.6323 - val_loss: 0.9044 - val_root_mean_squared_error: 0.9510
Epoch 00018: early stopping
15
Epoch 1/1000
37/37 [==============================] - 14s 264ms/step - loss: 0.0054 - root_mean_squared_error: 0.6579 - val_loss: 0.8963 - val_root_mean_squared_error: 0.9467
Epoch 2/1000
37/37 [==============================] - 9s 250ms/step - loss: 0.0055 - root_mean_squared_error: 0.6220 - val_loss: 0.9522 - val_root_mean_squared_error: 0.9758
Epoch 3/1000
37/37 [==============================] - 9s 250ms/step - loss: 0.0056 - root_mea

37/37 [==============================] - 10s 268ms/step - loss: 0.0056 - root_mean_squared_error: 0.6401 - val_loss: 0.8982 - val_root_mean_squared_error: 0.9477
Epoch 6/1000
37/37 [==============================] - 10s 265ms/step - loss: 0.0046 - root_mean_squared_error: 0.5921 - val_loss: 0.9620 - val_root_mean_squared_error: 0.9808
Epoch 7/1000
37/37 [==============================] - 10s 267ms/step - loss: 0.0052 - root_mean_squared_error: 0.6263 - val_loss: 0.9379 - val_root_mean_squared_error: 0.9685
Epoch 8/1000
37/37 [==============================] - 10s 261ms/step - loss: 0.0055 - root_mean_squared_error: 0.6339 - val_loss: 0.9891 - val_root_mean_squared_error: 0.9945
Epoch 9/1000
37/37 [==============================] - 9s 253ms/step - loss: 0.0058 - root_mean_squared_error: 0.6431 - val_loss: 0.9834 - val_root_mean_squared_error: 0.9917
Epoch 10/1000
37/37 [==============================] - 9s 255ms/step - loss: 0.0048 - root_mean_squared_error: 0.6062 - val_loss: 0.9126 - 

37/37 [==============================] - 9s 250ms/step - loss: 0.0059 - root_mean_squared_error: 0.6339 - val_loss: 0.9486 - val_root_mean_squared_error: 0.9740
Epoch 5/1000
37/37 [==============================] - 9s 237ms/step - loss: 0.0052 - root_mean_squared_error: 0.6260 - val_loss: 0.9117 - val_root_mean_squared_error: 0.9548
Epoch 6/1000
37/37 [==============================] - 9s 247ms/step - loss: 0.0043 - root_mean_squared_error: 0.5920 - val_loss: 0.9421 - val_root_mean_squared_error: 0.9706
Epoch 00006: early stopping
33
Epoch 1/1000
37/37 [==============================] - 15s 286ms/step - loss: 0.0055 - root_mean_squared_error: 0.6237 - val_loss: 0.8977 - val_root_mean_squared_error: 0.9475
Epoch 2/1000
37/37 [==============================] - 10s 270ms/step - loss: 0.0045 - root_mean_squared_error: 0.5886 - val_loss: 0.8761 - val_root_mean_squared_error: 0.9360
Epoch 3/1000
37/37 [==============================] - 9s 256ms/step - loss: 0.0053 - root_mean_squared_error: 

Epoch 7/1000
37/37 [==============================] - 10s 258ms/step - loss: 0.0056 - root_mean_squared_error: 0.6099 - val_loss: 0.9328 - val_root_mean_squared_error: 0.9658
Epoch 8/1000
37/37 [==============================] - 9s 255ms/step - loss: 0.0060 - root_mean_squared_error: 0.6233 - val_loss: 0.9318 - val_root_mean_squared_error: 0.9653
Epoch 9/1000
37/37 [==============================] - 10s 262ms/step - loss: 0.0054 - root_mean_squared_error: 0.6456 - val_loss: 0.9402 - val_root_mean_squared_error: 0.9696
Epoch 10/1000
37/37 [==============================] - 9s 255ms/step - loss: 0.0046 - root_mean_squared_error: 0.6111 - val_loss: 0.9387 - val_root_mean_squared_error: 0.9689
Epoch 11/1000
37/37 [==============================] - 10s 260ms/step - loss: 0.0044 - root_mean_squared_error: 0.6038 - val_loss: 0.9134 - val_root_mean_squared_error: 0.9557
Epoch 00011: early stopping
38
Epoch 1/1000
37/37 [==============================] - 14s 268ms/step - loss: 0.0053 - root_mea

37/37 [==============================] - 10s 264ms/step - loss: 0.0047 - root_mean_squared_error: 0.6081 - val_loss: 0.9412 - val_root_mean_squared_error: 0.9702
Epoch 00006: early stopping
48
Epoch 1/1000
37/37 [==============================] - 15s 287ms/step - loss: 0.0043 - root_mean_squared_error: 0.5984 - val_loss: 0.9183 - val_root_mean_squared_error: 0.9583
Epoch 2/1000
37/37 [==============================] - 10s 264ms/step - loss: 0.0051 - root_mean_squared_error: 0.6285 - val_loss: 0.8693 - val_root_mean_squared_error: 0.9324
Epoch 3/1000
37/37 [==============================] - 10s 260ms/step - loss: 0.0053 - root_mean_squared_error: 0.6266 - val_loss: 0.9097 - val_root_mean_squared_error: 0.9538
Epoch 4/1000
37/37 [==============================] - 10s 269ms/step - loss: 0.0051 - root_mean_squared_error: 0.6232 - val_loss: 0.9085 - val_root_mean_squared_error: 0.9531
Epoch 5/1000
37/37 [==============================] - 9s 255ms/step - loss: 0.0055 - root_mean_squared_erro

In [ ]:
# sketch1 = sketch1.astype({"text": str})
# text = list(sketch1['text'])

# from keras import backend as K
# sec_last = model.layers[3]
# get_output = K.function([model.layers[0].input, ],[sec_last.output, model.layers[-1].output])
# [emb_text, predictions] = get_output(tf.constant(text))

# np.savetxt('text emb 1-'+var, emb_text)

In [ ]:
# ##to test which BEET
# idx = 0
# output = pd.DataFrame()
# for key in map_name_to_handle.keys():
#     bert_model_name = key
#     tfhub_handle_encoder = map_name_to_handle[bert_model_name]
#     tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]   
#     text_input = Input(shape=(), dtype=tf.string, name='text')
#     preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
#     encoder_inputs = preprocessing_layer(text_input)
#     encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
#     outputs = encoder(encoder_inputs)
#     net = outputs['pooled_output']
#     # net = layers.Dropout(0.1)(net)
#     net = layers.Dense(128, activation='relu')(net)
#     net = layers.Dropout(0.1)(net)
#     out1y = layers.Dense(1, activation='relu', name='out1y')(net)
#     model = Model(text_input, out1y)

#     model.compile(optimizer=optimizer,
#                   loss=tf.keras.losses.MeanSquaredError(),
#                   metrics= tf.keras.metrics.RootMeanSquaredError())
#     training_history = model.fit(
#         x = tf.constant(train_text),
#         y = {'out1y':np.array(train_cost)},
#         sample_weight={'out1y': np.array(weights)},
#         epochs=epochs,
#         batch_size = batch_size,
#         steps_per_epoch=steps_per_epoch,
#         validation_data=(
#             tf.constant(val_text),
#             {'out1y':np.array(val_cost)}),
#         callbacks=[early_stopping_callback]
#     )

#     test_results0 = model.predict(tf.constant(test_text))
#     test_result= test_results0.reshape(len(test_text)).tolist()
#     test_cost = test_df[var]

#     difference_array = np.subtract(test_result, test_cost)
#     squared_array = np.square(difference_array)
#     mse = squared_array.mean()

#     correlation_matrix = np.corrcoef(test_cost, test_result)
#     correlation_xy = correlation_matrix[0,1]
#     r_squared = correlation_xy**2

#     output.loc[idx,'n'] = idx
#     output.loc[idx,'var'] = var
#     output.loc[idx,'lr'] = init_lr
#     output.loc[idx,'mse'] = mse
#     output.loc[idx,'r^2'] = r_squared
#     output.loc[idx,'model'] = key
#     idx += 1
#     print (key+":  ", r_squared)